In [1]:
import pandas as pd
import numpy as np
# import yfinance
import tqdm

# from mplfinance import candlestick_ohlc
from mplfinance.original_flavor import candlestick_ohlc

import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go

from datetime import datetime

# https://github.com/elcreek/machinelearning/blob/master/Support_and_resistance.ipynb

In [2]:
def isSupport(df,i):
  # fractal (i-2)>(i-1)>i<(i+1)<(i+2)
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

  return resistance

In [3]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [4]:
def plot_all(df, level):

    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

    # fig = go.Figure(data=[go.Candlestick(x=df['Date'],
    #             open=df['Open'],
    #             high=df['High'],
    #             low=df['Low'],
    #             close=df['Close'])])
    fig = go.Figure(data=candlestick)

    fig.add_hline(level)

    # for level in levels:
    #     fig.add_shape(type='line',
    #             x0=df['Date'][level[0]],
    #             y0=level[1],
    #             x1=max(df['Date']),
    #             y1=level[1],
    #             line=dict(color='black',),
    #             xref='x',
    #             yref='y')
    
    fig.layout.xaxis.rangeslider.visible = False


    fig.show()

In [5]:

name = 'BTC-USD'
dataframes={}
timeframe='1d'
outdir2 = f"{os.getcwd()}/data/{timeframe}"


for filename in os.listdir(outdir2):
    symbol = filename.split("_")[0]
    df = pd.read_csv(f"{outdir2}/{filename}",index_col=['Time'], parse_dates=['Time'])

    df['Date'] = pd.to_datetime(df.index)
    # change datetime into matplotlib time
    # does not work with plotly
    # df['Date'] = df['Date'].apply(mpl_dates.date2num)

    df = df.loc[:,['Date' ,'Open', 'High', 'Low', 'Close']]

    levels = []
    # start from 2 because i need i-2
    # end at shape[0]-2 because  i need i+2
    # i is the number of current row
    for i in range(2,df.shape[0]-2):
        if isSupport(df,i):
            levels.append((i,df['Low'][i]))
        elif isResistance(df,i):
            levels.append((i,df['High'][i]))


    s =  np.mean(df['High'] - df['Low'])

    levels = []
    for i in range(2,df.shape[0]-2):
        if isSupport(df,i):
            l = df['Low'][i]

            if isFarFromLevel(l):
                levels.append((i,l))

        elif isResistance(df,i):
            l = df['High'][i]

            if isFarFromLevel(l):
                levels.append((i,l))

    for level in levels:
        if (df['Close'][-2] < level[1]) and (df['Close'][-1] > level[1]):
            print(f"{level[1]} breakout {symbol}")
    
    dataframes[symbol] = df



4.367 breakout ACMUSDT
0.080458 breakout ANKRUSDT
21.7 breakout ANYUSDT
36.165 breakout ARUSDT
1.169 breakout AVAUSDT
76.27 breakout AVAXUSDT
0.661 breakout BAKEUSDT
12.91 breakout BALUSDT
0.8412 breakout BATUSDT
297.0 breakout BCHUSDT
0.14365 breakout BLZUSDT
388.28 breakout BNBUSDT
9.89 breakout BONDUSDT
40100.0 breakout BTCUSDT
37850.0 breakout BTCUSDT
0.05304 breakout CELRUSDT
0.5389 breakout CHRUSDT
9.88 breakout CITYUSDT
3.173 breakout CRVUSDT
1.3499 breakout CTKUSDT
0.46697 breakout CTSIUSDT
0.3035 breakout CVCUSDT
95.3 breakout DASHUSDT
0.0027847 breakout DENTUSDT
0.1082 breakout DFUSDT
0.02175 breakout DGBUSDT
0.04473 breakout DOCKUSDT
0.72 breakout DREPUSDT
1.72 breakout ENJUSDT
5.322 breakout ERNUSDT
20.6023 breakout FILUSDT
0.0977 breakout FIOUSDT
0.2476 breakout FLMUSDT
43.0 breakout FTTUSDT
45.09 breakout FTTUSDT
0.01037 breakout FUNUSDT
303.5 breakout GNOUSDT
26.73 breakout HNTUSDT
0.0048 breakout HOTUSDT
598.0 breakout ILVUSDT
0.02688 breakout IOSTUSDT
0.07762 breakout 

In [11]:
# wont work correctly because ther is the level variable inside plot_all function 
# wich he belongs only to the last df, so the solution is to remove
# shape method from plot_all function
df = dataframes['XLMUSDT']
plot_all(df,0.20878)